# Make AAdict 

In [3]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io
from datetime import datetime
import time

Just run the following cell to open AAdict

In [4]:
# To Reload AAdict without rerunning entire notebook, run this cell
#AAdict = pickle.load(open('AAdict','rb'))

In [5]:
# Read in Academy Awards df (AAdf)
AAdf = pd.read_excel("Academy_Awards_2006.xls")

# Update df
# Concat Sort Title with first part of year; store in "title"
# This will be helpful when using ia.search_movie function
AAdf['Year'] = AAdf['Year'].values.astype(str)
AAdf['yr'] = AAdf.apply( lambda row: row['Year'][:4],axis=1 )
AAdf['titleyr'] = AAdf.apply( lambda row: '%s (%s)' % (row['Sort Title'],row['Year'][:4]),axis=1)
# Convert 'Winner' "X" to 1
AAdf['Winner?'] = 1*(AAdf['Winner?'] == 'X')
## TO DO
### Convert 'Country' "0" to USA
# Convert 'Sort Title' to be strings (easier for comparison later)
#AAdf['Sort Title'] = str(AAdf['Sort Title'])
# If movie title is "[no specific film title]", delete
AAdf = AAdf[AAdf['Sort Title'] != "[no specific film title]"].copy()

# Subset DF
AAdf = AAdf[AAdf["yr"]>="1985"].copy()

# Store all possible awards in "awards" list
# This will be used when making aadict to indicate which awards the movie was nominated for/won
awards = list(set(list(AAdf['Category'])))

# View head of AAdf
AAdf.head()

,Original Title,English Title,Sort Title,Year,Country,Award,Category,Winner?,Nominee(s),__ Academy Awards,Item,yr,titleyr
6356,The Color Purple,The Color Purple,Color Purple,1985,0,Academy Award,Best Picture,0,"Steven Spielberg, Kathleen Kennedy, Frank Mars...",58th,6357,1985,Color Purple (1985)
6357,Kiss of the Spider Woman,Kiss of the Spider Woman,Kiss of the Spider Woman,1985,0,Academy Award,Best Picture,0,David Weisman (Producer),58th,6358,1985,Kiss of the Spider Woman (1985)
6358,Out of Africa,Out of Africa,Out of Africa,1985,0,Academy Award,Best Picture,1,Sydney Pollack (Producer),58th,6359,1985,Out of Africa (1985)
6359,Prizzi's Honor,Prizzi's Honor,Prizzi's Honor,1985,0,Academy Award,Best Picture,0,John Foreman (Producer),58th,6360,1985,Prizzi's Honor (1985)
6360,Witness,Witness,Witness,1985,0,Academy Award,Best Picture,0,Edward S. Feldman (Producer),58th,6361,1985,Witness (1985)


In [52]:
### Movie Attribute functions ##

# Function:  get_releasedate
# Purpose:  get the USA release date given a movieobj
# Parameters: 
    # movieobj:  a single IMDBmovie object
# Returns: the USA release date of the movie as a class datetime
def get_releasedate(movieobj):
    try:
        ia.update(movieobj, 'release dates')
        date = str(movieobj.data['release dates']).split("USA::", 1)[1]
        day = str(date.split(" ")[0])
        month = str(date.split(" ")[1])
        year = str(date.split(" ")[2].split("'")[0])
        releasedate = datetime.strptime(year + "-" + month + "-" + day, "%Y-%B-%d").date()
    except:
        releasedate = np.nan
    return releasedate

# Function:  get_mpaa
# Purpose:  get the mpaa rating given a movieobj
# Parameters: 
    # movieobj:  a single IMDBmovie object
# Returns: the USA release date of the movie as a class datetime
def get_mpaa(movieobj):
    try:
        mpaa = str(movieobj.data['mpaa']).split("Rated ", 1)[1].split(" ")[0]
    except:
        mpaa = np.nan
    return mpaa

# Function:  get_genres
# Purpose:  get the list of genres given in a movieobj
# Parameters:
    # movieobj: a single IMDBmovie object
# Returns:  a list of genres
def get_genres(movieobj):
    try:
        genres = movieobj.data['genres']
    except:
        genres = np.nan
    return genres

# Function:  get_runtime
# Purpose:  get the runtime given in a movieobj
# Parameters:
    # movieobj: a single IMDBmovie object
# Returns:  USA runtime
def get_runtime(movieobj):
    try:
        runtime = movieobj.data['runtimes'][0]
        try:
            runtime = int(runtime)
        except:
            try:
                runtime = int(runtime.split(':')[0])
            except:
                try:
                    runtime = int(runtime.split(':')[1])
                except:
                    runtime = int(runtime.split(':')[2])
    except:
        runtime = 0
    return runtime

# Function:  get_cast
# Purpose:  get the list of genres given in a movieobj
# Parameters:
    # movieobj: a single IMDBmovie object
# Returns:  a list of cast
def get_cast(movieobj):
    try:
        castdata = movieobj.data['cast']
        cast = []
        for person in castdata:
            cast.append(person.personID)
    except:
        cast = []
    return cast

# Function:  get_director
# Purpose:  get the list of directors given in a movieobj
# Parameters:
    # movieobj: a single IMDBmovie object
# Returns:  a list of directors
def get_director(movieobj):
    try:
        directordata = movieobj.data['director']
        director = []
        for person in directordata:
            director.append(person.personID)
    except:
        director = []
    return director

# Function:  get_keywords
# Purpose:  get the list of keywords given in a movieobj
# Parameters:
    # movieobj: a single IMDBmovie object
# Returns:  a list of keywords
def get_keywords(movieobj):
    try:
        ia.update(movieobj, 'keywords')
        keywords = movieobj.data['keywords']
    except:
        keywords = np.nan
    return keywords

In [53]:
# Function:  find_movie
# Purpose: to sort through possible IMDB movie objects and find just one
# Parameters:  
    #title: title of movie
    #year:  year of movie
    #mlist: list of possible IMDB movie objects
# Returns movieobj
def find_movie(title, year,  mlist):
    # find movies that came out in the same year                                                                                                                                    
    year_list = []
    for movie in mlist:
        try:
            if movie.data['year'] == int(year):
                year_list.append(movie)
            # else see if one - two years off
            elif movie.data['year'] == int(year) + 1:
                year_list.append(movie)
            elif movie.data['year'] == int(year) - 1:
                year_list.append(movie)
            elif movie.data['year'] == int(year) + 2:
                year_list.append(movie)
            elif movie.data['year'] == int(year) - 2:
                year_list.append(movie)
        except:
            pass
    # if the years do not match, there is no match                                                                                                                                  
    if len(year_list) < 1:
        return None
    if len(year_list) == 1:
        return ia.get_movie(year_list[0].movieID)
    else:
        # process the title                                                                                                                                                         
        sorted_title = "".join(sorted(title)).replace(" ", "")
        len_sorted_title = len(sorted_title)
        # check whether movies that came out in the same year                                                                                                                       
        # have the same letters                                                                                                                                                     
        counts = [0]*len(year_list)
        for j in range(len(year_list)):
            mtitle = year_list[j]['title']
            sorted_mtitle = "".join(sorted(mtitle)).replace(" ", "")
            if len_sorted_title == len(sorted_mtitle):
                # if the title cannot be converted to a string                                                                                                                      
                # it is not the correct title                                                                                                                                       
                try:
                    sorted_mtitle = str(sorted_mtitle)
                except:
                    continue
                for i in range(len_sorted_title):
                    if sorted_title[i] == sorted_mtitle[i]:
                        counts[j] += 1
            else:
                continue
        k = counts.index(max(counts))
        if len(year_list) >= 1:
            #return year_list[k]
            return ia.get_movie(year_list[k].movieID)

In [54]:
# Function:  find_movieobj
# Purpose:  To convert a tuple of movie's information into one IMDB movie object
# Parameters:  
    # movie_tuple:  a tuple in the format (English Title, Sort Title, English Title + yr, yr)
# Returns: IMDB movie object
def find_movieobj(movie_tuple):   
    ## Step 1:  Find the IMDB movie object ("movieobj")
    arg1 = movie_tuple[0]        # English Title (1st choice for arg1)
    if type(arg1) == int:        # check if movie title is an int, if so convert to string
        arg1 = str(arg1)
    arg2 = movie_tuple[3]        # movie year
    arg3 = ia.search_movie(arg1)       # list of possible movies
    movieobj = find_movie(arg1, arg2, arg3)  # find IMDB movie object ("movieobj") using "find_movie" function
    if movieobj == None:                     # if returned none, try again using title +  yr search list of movies & English Title
        arg1alt = movie_tuple[1]             # Non-English Title (alternate choice if arg1 fails)
        if type(arg1alt) == int:             # check if movie title is an int, if so convert to string
            arg1alt = str(arg1alt)
        arg3alt1 = ia.search_movie(arg1alt)              # list of possible movies searching for title + yr (alt choice to arg3)
        movieobj = find_movie(arg1, arg2, arg3alt1)
    if movieobj == None:                     # if returned none, try again using title +  yr search list of movies & Non-English Title
        arg3alt2 = ia.search_movie(movie_tuple[2]) # list of possible movies searching for Non-English title (alt choice to arg3)
        find_movie(arg1alt, arg2, arg3alt2)
    if movieobj == None:                     # if returned none, try again using Non-English search list of movies & English Title
        movieobj = find_movie(arg1, arg2, arg3alt1)
    if movieobj == None:                     # if returned none, try again using Non-English search list of movies & Non-English Title
        movieobj = find_movie(arg1alt, arg2, arg3alt1)
    return movieobj

In [56]:
# Function:  make_moviedict
# Purpose:  To convert movieobj and movie_tuple into a dictionary within AAdict
# Parameters:  
    # movieobj:  a single IMDBmovie object
    # movie_tuple:  a tuple in the format (English Title, Sort Title, English Title + yr, yr)
# Returns None (if no movieobj) or 1 if successfully appended dictionary to AAdict
def make_moviedict(movieobj, movie_tuple):
    if movieobj is None:
        return None
    else:
        ## Get movie id ##
        movid = movieobj.movieID
        ## Populate dictionary, main key is movie id ##
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = movie_tuple[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==movie_tuple[0]]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==movie_tuple[0]) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==movie_tuple[0]]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==movie_tuple[0]]['Country'])[0]
        # "releasedate": USA movie release date in form yyyy-mm-dd
        AAdict[movid]['releasedate'] = get_releasedate(movieobj)
        # "mpaa": mpaa rating for the movie (i.e. R, PG-13, PG, G)
        AAdict[movid]['mpaa'] = get_mpaa(movieobj)
        # "genres": list of genres
        AAdict[movid]['genres'] = get_genres(movieobj)
        # "runtime": USA runtime
        AAdict[movid]['runtime'] = get_runtime(movieobj)
        # "cast": movie cast
        AAdict[movid]['cast'] = get_cast(movieobj)
        # "director": list of directors
        AAdict[movid]['director'] = get_director(movieobj)
        # "keywords": list of keywords
        AAdict[movid]['keywords'] = get_keywords(movieobj)
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            if AAdict[movid]["Nominated %s" %award] == True:
                AAdict[movid]["Nominated %s" %award] = list(AAdf[(AAdf['English Title']==movie_tuple[0]) & (AAdf['Category']==award)]['Nominee(s)'])[0]
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
            if AAdict[movid]["Won %s" %award] == True:
                AAdict[movid]["Won %s" %award] = list(AAdf[(AAdf['English Title']==movie_tuple[0]) & (AAdf['Category']==award)]['Nominee(s)'])[0]
        return 1

Note:  You can skip the next several cells and just run the next cell (pickle.load) to get the complete moviedict
I ended up doing this in 2 pieces b/c it took long to run on the full file and would sometimes timeout.

In [57]:
# Prep to create Academy Awards Dictionary ("AAdict"), a dict of dicts
# AAdict keyed by IMDB movie IDs
# Each movie id dict has keys containing information about the movie & Academy Award information

# Get a list of the unique movies in Academy Awards DF ("AAdf")
# Store the English Title, Sort Title, English Title + yr, and yr in "AAuniquemovies"
#AAuniquemovies = list(set(zip(AAdf['English Title'], AAdf['Sort Title'], AAdf['titleyr'], AAdf['yr'])))
AAuniquemovies = list(set(zip(AAdf['English Title'], AAdf['Sort Title'], AAdf['titleyr'], AAdf['yr'])))

# Create empty AAdict
AAdict = {}

# Keep track of movies that failed to find a IMDB movie object (i.e. movieobj = None)
AAmissingmovies = list()

In [58]:
%%time

for i in range(len(AAuniquemovies)):
    ## STEP 1:  Get movieobj of movie using get_movieobj
    movieobj = find_movieobj(AAuniquemovies[i])
    ## STEP 2:  Append to AAdict using make_moviedict
    success = make_moviedict(movieobj, AAuniquemovies[i])
    if success is None:
        AAmissingmovies.append(AAuniquemovies[i])
    print i

0
1
2
3
4
5
6
7
8
9
CPU times: user 1.7 s, sys: 341 ms, total: 2.04 s
Wall time: 27.2 s


### Check
Check to see what movies are missing from AAdict compared to the unique movies from AAdf

In [14]:
# CHECK
print "number of movies in AAdict:", len(AAdict.keys())
print "number of movies in AAdf:", len(AAuniquemovies)
    # inconsistent numbers...
    
## DEBUG ##
# Get list of what is in the dictionary & what movies returned "None"
debuglist = list()
for key in AAdict.keys():
    debuglist.append((AAdict[key]['title'], AAdict[key]['year']))

# Compare to list of movies from AAdf
print "list of movies missing from AAdict:"
missing = list(set(AAuniquemovies) -set(debuglist))
print len(missing)
missing

number of movies in AAdict: 4158
number of movies in AAdf: 4209
list of movies missing from AAdict:
119


[(u'Song without End (The Story of Franz Liszt)', '1960'),
 (u'Amarcord', '1975'),
 (u'Van Gogh', '1949'),
 (u'First Love', '1970'),
 (u'The Old Man and the Sea', '1999'),
 (u"My Night at Maud's", '1970'),
 (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)',
  '1946'),
 (u'Sweet and Lowdown', '1999'),
 (u'Men in Black', '1997'),
 (u'A Voice Is Born: The Story of Niklos Gafni', '1947'),
 (u"Jacques-Yves Cousteau's World Without Sun", '1964'),
 (u'The Time Machine', '2002'),
 (u'The Forbidden Volcano', '1966'),
 (u'Good Fellas', '1990'),
 (u'Skyscraper', '1959'),
 (u'A Farewell to Arms', '1957'),
 (u'War and Peace', '1968'),
 (u'In Which We Serve', '1943'),
 (u'Hamlet', '1990'),
 (u'The War of the Worlds', '1953'),
 (u'Heaven Can Wait', '1978'),
 (u'The Champ', '1979'),
 (u'Investigation of a Citizen Above Suspicion', '1971'),
 (u'Day for Night', '1974'),
 (u'Contact', '1997'),
 (u'Richard III', '1995'),
 (u'Marriage Italian Style', '1965'),
 (u'Goodbye, Mr. Chips'

###Debug

####DEBUG, STEP 1
Adjust 1 year up/down and research; Save the remaining missing movies in AAmissingmovies1

In [29]:
%%time

AAmissingmovies1 = list()
for missingmovie in missing:
    movie = find_movie( missingmovie[0], missingmovie[1], ia.search_movie(missingmovie[0]))
    if movie is None:
        movie = find_movie( missingmovie[0], int(missingmovie[1]) - 1, ia.search_movie(missingmovie[0]))
    if movie is None:
        movie = find_movie( missingmovie[0], int(missingmovie[1]) + 1, ia.search_movie(missingmovie[0]))
    if movie is not None:
        movid = movie.movieID
        title = missingmovie[0]
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['English Title']==title]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==title]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==title) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==title]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==title]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])
        print ("True", missingmovie, movie)
    else:
        AAmissingmovies1.append(missingmovie)
        print ("False", missingmovie, movie)

('False', (u'Song without End (The Story of Franz Liszt)', '1960'), None)
('True', (u'Amarcord', '1975'), <Movie id:2038256[http] title:_Diario segreto di Amarcord (1974)_>)
('True', (u'Van Gogh', '1949'), <Movie id:0040930[http] title:_Van Gogh (1948)_>)
('True', (u'First Love', '1970'), <Movie id:0065703[http] title:_First Love (1970)_>)
('True', (u'The Old Man and the Sea', '1999'), <Movie id:0207639[http] title:_The Old Man and the Sea (1999)_>)
('True', (u"My Night at Maud's", '1970'), <Movie id:0064612[http] title:_My Night at Maud's (1969)_>)
('False', (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)', '1946'), None)
('True', (u'Sweet and Lowdown', '1999'), <Movie id:0158371[http] title:_Sweet and Lowdown (1999)_>)
('False', (u'Men in Black', '1997'), None)
('False', (u'A Voice Is Born: The Story of Niklos Gafni', '1947'), None)
('False', (u"Jacques-Yves Cousteau's World Without Sun", '1964'), None)
('True', (u'The Time Machine', '2002'), <Movie id:026869

In [33]:
print "Movies still missing:", len(AAmissingmovies1)

Movies still missing: 26


#### DEBUG, STEP 2
Search using non-English title; Save the remaining missing movies in AAmissingmovies2

In [36]:
%%time

# See if non English Title is searchable
AAmissingmovies2 = list()
for movie in AAmissingmovies1:
    movieobj = None
    movietitle = list(AAdf[AAdf['English Title']==movie[0]]['Sort Title'])[0]
    movieyr = list(AAdf[AAdf['English Title']==movie[0]]['yr'])[0]
    movieobj = ia.search_movie((movietitle + " (%s)" %movieyr))
    if movieobj is None:
        movieyr = int(movieyr)
        movieobj = find_movie(movietitle, movieyr - 1, ia.search_movie(movietitle))
    if movieobj is None:
        movieobj = find_movie(movietitle, movieyr + 1, ia.search_movie(movietitle))
    if len(movieobj) >= 1:
        movieobj = movieobj[0]
        movid = movieobj.movieID
        AAdict[movid] = {}
        # "title": title of movie
        AAdict[movid]['title'] = list(AAdf[AAdf['Sort Title']==movietitle]['English Title'])[0]
        # "nominations": list of Oscar nominations
        AAdict[movid]['nominations'] = list(AAdf[AAdf['Sort Title']==movietitle]['Category'])
        # "won": list of Oscars won
        AAdict[movid]['won'] = list(AAdf[(AAdf['Sort Title']==movietitle) & (AAdf['Winner?']==1)]['Category'])
        # "year": year Oscar won
        AAdict[movid]['year'] = list(AAdf[AAdf['Sort Title']==movietitle]['yr'])[0]
        # "country": country of movie
        AAdict[movid]['country'] = list(AAdf[AAdf['Sort Title']==movietitle]['Country'])[0]
        # make each award individual key and the value to indicate whether movie won/nominated or not
        # Loop through awards list and indicate if movie was nominated or won
        for award in awards:
            # "Nominated award_name": True or False
            AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
            # "Nominated award_name": True or False
            AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])   
        print ("True", movie, movieobj)
    else:
        AAmissingmovies2.append(movie)
        print ("False", movie, movieobj)

('True', (u'Song without End (The Story of Franz Liszt)', '1960'), <Movie id:0054324[http] title:_Song Without End (1960)_>)
('True', (u'Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)', '1946'), <Movie id:0038825[http] title:_Paramount News Issue #37 (1946)_>)
('True', (u'Men in Black', '1997'), <Movie id:0025488[http] title:_Men in Black (1934)_>)
('False', (u'A Voice Is Born: The Story of Niklos Gafni', '1947'), [])
('True', (u"Jacques-Yves Cousteau's World Without Sun", '1964'), <Movie id:0058364[http] title:_World Without Sun (1964)_>)
('True', (u'The Forbidden Volcano', '1966'), <Movie id:0061163[http] title:_Le volcan interdit (1966)_>)
('True', (u'War and Peace', '1968'), <Movie id:0049934[http] title:_War and Peace (1956)_>)
('True', (u'Anastasia', '1997'), <Movie id:0048947[http] title:_Anastasia (1956)_>)
('True', (u'The Brothers Karamazov', '1969'), <Movie id:0051435[http] title:_The Brothers Karamazov (1958)_>)
('True', (u'Forbidden Games', '1954'), <

In [42]:
print "Movies still missing:", len(AAmissingmovies2)

Movies still missing: 9


#### DEBUG, STEP 3
Mannually search remaining movies

In [41]:
missingids = {}
missingids['A Voice Is Born: The Story of Niklos Gafni'] = '0039968'
missingids['Mighty Mouse in Gypsy Life'] = '0037912'
missingids['Cavalcade of Dance with Veloz and Yolanda'] = '0035724'
missingids['The Usual Unidentified Thieves'] = '0052216'
missingids["Eyes on the Prize: America's Civil Rights Years/Bridge to Freedom 1965"] = '0092999'
missingids['Sandokan No. 8'] = '0073654'
missingids["Screen Snapshots' 50th Anniversary of Motion Pictures"] = '0036576'
missingids['Mama Turns a Hundred'] = '0079517'
missingids['Rashomon'] = '0042876'

In [48]:
for movie in missingids:
    movid = missingids[movie]
    AAdict[movid] = {}
    # "title": title of movie
    AAdict[movid]['title'] = movie
    # "nominations": list of Oscar nominations
    AAdict[movid]['nominations'] = list(AAdf[AAdf['English Title']==movie]['Category'])
    # "won": list of Oscars won
    AAdict[movid]['won'] = list(AAdf[(AAdf['English Title']==movie) & (AAdf['Winner?']==1)]['Category'])
    # "year": year Oscar won
    AAdict[movid]['year'] = list(AAdf[AAdf['English Title']==movie]['yr'])[0]
    # "country": country of movie
    AAdict[movid]['country'] = list(AAdf[AAdf['English Title']==movie]['Country'])[0]
    # make each award individual key and the value to indicate whether movie won/nominated or not
    # Loop through awards list and indicate if movie was nominated or won
    for award in awards:
        # "Nominated award_name": True or False
        AAdict[movid]["Nominated %s" % award] = award in list(AAdict[movid]['nominations'])
        # "Nominated award_name": True or False
        AAdict[movid]["Won %s" % award] = award in list(AAdict[movid]['won'])        

In [ ]:
# To Save new AAdict, run this cell
#AAdict = pickle.load(open('AAdict','rb'))

In [7]:
AAdict['0120338']['won']

[u'Best Writing, Original Screenplay',
 u'Best Picture',
 u'Best Director',
 u'Best Music, Dramatic Score',
 u'Best Music, Song',
 u'Best Film Editing',
 u'Best Cinematography',
 u'Best Art Direction',
 u'Best Costume Design',
 u'Best Visual Effects',
 u'Best Sound',
 u'Best Sound Effects Editing']

In [452]:
#missingids = {}
#missingids['The Forbidden Volcano'] = '0061163'
#missingids['The Usual Unidentified Thieves'] = '0052216'
#missingids['Kate & Leopold'] = '0035423'
#missingids['Malcolm X'] = '0104797'
#missingids['Cavalcade of Dance with Veloz and Yolanda'] = '0035724'
#missingids['A Voice Is Born: The Story of Niklos Gafni'] = '0039968'
#missingids["Eyes on the Prize: America's Civil Rights Years/Bridge to Freedom 1965"] = '0092999'
#missingids['Sandokan No. 8'] = '0073654'
#missingids["Screen Snapshots' 50th Anniversary of Motion Pictures"] = '0036576'
#missingids['Song without End (The Story of Franz Liszt)'] = '0054324'
#missingids['Mighty Mouse in Gypsy Life'] = '0037912'
#missingids['The Sorrow and the Pity'] = '0066904'
#missingids['Mama Turns a Hundred'] = '0079517'
#missingids['Paramount News Issue #37 (Twentieth Anniversary Issue! 1927.....1947)'] = '0038825'
#missingids['Men in Black'] = '0119654'
#missingids["Jacques-Yves Cousteau's World Without Sun"] = '0058364'
#missingids['War and Peace'] = '0063794'
#missingids['Yes, Giorgio'] = '0084931'
#missingids['Rashomon'] = '0042876 '
#missingids['Henry V'] = '0036910'